In [ ]:
import os
import json
from pprint import pprint

In [ ]:
# Create the "data/cleaned" directory if it doesn't exist
if not os.path.exists("data/cleaned"):
    os.makedirs("data/cleaned")
if not os.path.exists("data/cleaned/sample"):
    os.makedirs("data/cleaned/sample")

## Businesses

In [ ]:
# Creates a new list object 'business' (loading in the JSON file)
with open("data/yelp_academic_dataset_business.json", "r") as f:
    businesses = [json.loads(line) for line in f]

In [ ]:
keys_to_keep = [
    "longitude",
    "name",
    "categories",
    "review_count",
    "stars",
    "latitude",
    "business_id",
]

In [ ]:
# Creates a new object called business that includes the attributes of the keys_to_keep list
for business in businesses:
    for key in list(business.keys()):
        if key not in keys_to_keep:
            business.pop(key)

In [ ]:
with open("data/cleaned/businesses.json", "w") as f:
    f.write('')
with open("data/cleaned/businesses.json", "a") as f:
    for business in businesses:
        f.write(json.dumps(business) + "\n")

## Reviews

In [ ]:
with open("data/yelp_academic_dataset_review.json", "r") as f:
    reviews = [json.loads(line) for line in f] # List syntax that creates a new list

In [ ]:
keys_to_keep = [
    "business_id",
    "date",
    "review_id",
    "stars",
    "text",
    "user_id",
]

In [ ]:
for review in reviews:
    for key in list(review.keys()):
        if key not in keys_to_keep:
            review.pop(key)

In [ ]:
with open("data/cleaned/reviews.json", "w") as f: # creates a new empty file
    f.write('')
with open("data/cleaned/reviews.json", "a") as f: # writes each review as a new line
    for review in reviews:
        f.write(json.dumps(review) + "\n")

## Users

In [ ]:
with open("data/yelp_academic_dataset_user.json", "r") as f:
    users = [json.loads(line) for line in f]

In [ ]:
keys_to_keep = [
    "average_stars",
    "friends",
    "name",
    "review_count",
    "user_id",
]

In [ ]:
for user in users:
    for key in list(user.keys()):
        if key not in keys_to_keep:
            user.pop(key)

Now removing users that don't have friends

In [ ]:
# Change the friends property to a list instead of a string
for user in users:
    user["friends"] = user["friends"].split(", ")

In [ ]:
# Remove users that don't have friends
users = [user for user in users if len(user["friends"]) > 0 and user["friends"] != ["None"]]

In [ ]:
with open("data/cleaned/users.json", "w") as f:
    f.write('')
with open("data/cleaned/users.json", "a") as f:
    for user in users:
        f.write(json.dumps(user) + "\n")